# Predict Future Sales

## How to

You don't have to do anything special, pretrained models are provided. Simply clone repo, run this notebook and enjoy. 

Can send result_lgb+lr.csv file for evaluation

## Requirements

Notebook was created in typical anaconda environment

Uncomment and run cell below. to install lightgbm (if needed)

In [1]:
#!pip install lightgbm

In [2]:
import pathlib
import json
import pandas as pd
import numpy as np
import random
from itertools import product
import gc
import tqdm.notebook as tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, PredefinedSplit
import lightgbm as lgb
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from scipy.sparse import hstack, vstack
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
import time
start = time.time()


## Utility methods

In [3]:
random.seed(0)
def downcast_dtypes(df):    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

def concat_df(train_data, test_data):
    return pd.concat([train_data, test_data], sort=True).reset_index(drop=True), train_data.shape[0]

def divide_df(all_data, train_size):
    if 'target' in all_data:
        return all_data.loc[:train_size-1], all_data.loc[train_size:].drop(['target'], axis=1)
    return all_data.loc[:train_size-1], all_data.loc[train_size:]

def get_result_df(predict):
    result = pd.DataFrame(data={'ID': range(0, 214200), 'item_cnt_month': predict})
    result['item_cnt_month'] = result['item_cnt_month'].clip(0, 20)
    return result

def get_mix(alpha, X):
    return (alpha * X[:,0]) + ((1-alpha) * X[:,1])

def get_best_alpha(X_train_level2, target):
    alphas_to_try = np.linspace(0, 1, 1001)
    min_mse = 100
    best_alpha = 1
    
    for alpha in alphas_to_try:
        mix = get_mix(alpha, X_train_level2)
        mse = mean_squared_error(target, mix)
        if min_mse > mse:
            min_mse = mse
            best_alpha = alpha
    return best_alpha

# this factor is needed for pipeline testing, make < 1 to reduce amount of data in work
cv_fraction = 1
def get_items_subset(X_train): 
    item_set = X_train['item_id'].unique()
    #random.shuffle(item_set)
    l = int(len(item_set) * cv_fraction)
    cv_item_set = item_set[:l]
    return X_train['item_id'].isin(cv_item_set)

## Data read

In [4]:
df_train = pd.read_csv('sales_train.csv')
df_test = pd.read_csv('test.csv')
df_shops = pd.read_csv('shops.csv')
df_items = pd.read_csv('items.csv')
df_item_cats = pd.read_csv('item_categories.csv')

In [5]:
ENABLE_PRICE_STATS = False
enable_subtype_code_mean = False

## Feature engineering

In [6]:
# removing item_cnt_day outliers
def remove_outlier(df_train, col):
    Q1 = df_train[col].quantile(0.25)
    Q3 = df_train[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = (df_train[col] > Q3+1.5*IQR)
    df_train[outliers].shape
    df_train.drop(df_train[outliers].index, inplace=True)
#remove_outlier(df_train, 'item_cnt_day')
#remove_outlier(df_train, 'item_price')

In [7]:
# ADD MONTH why not added??? -bad result (not one hot encoded) least for trees

# can be good of LR
#df_train['month'] = pd.to_datetime(df_train['date'], format='%d.%m.%Y').dt.month
# TODO add week, then add mean by week?

# FIX SHOPS

# Якутск Орджоникидзе, 56
df_train.loc[df_train.shop_id == 0, 'shop_id'] = 57
df_test.loc[df_test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
df_train.loc[df_train.shop_id == 1, 'shop_id'] = 58
df_test.loc[df_test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
df_train.loc[df_train.shop_id == 10, 'shop_id'] = 11
df_test.loc[df_test.shop_id == 10, 'shop_id'] = 11

In [8]:
# Create igem categories and cities from shops

# Extract type and sub type code
df_item_cats['split'] = df_item_cats['item_category_name'].str.split('-')
df_item_cats['type'] = df_item_cats['split'].map(lambda x: x[0].strip())
df_item_cats['type_code'] = LabelEncoder().fit_transform(df_item_cats['type'])
# if subtype is nan then type
df_item_cats['subtype'] = df_item_cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
df_item_cats['subtype_code'] = LabelEncoder().fit_transform(df_item_cats['subtype'])

# features = 10
# tfidf = TfidfVectorizer(max_features=features)
# df_item_cats['item_category_name'] = df_item_cats['item_category_name'].replace('[0-9!"\?\.)(,\+\*\[\]/:\-\'&]', ' ', regex=True)
# txtFeatures = pd.DataFrame(tfidf.fit_transform(df_item_cats['item_category_name']).toarray())
# cols = txtFeatures.columns

df_item_cats = df_item_cats[['item_category_id','type_code', 'subtype_code']]
# for i in range(features):
#     df_item_cats['item_category_tfidf_' + str(i)] = txtFeatures[cols[i]]

# Extract city
df_shops.loc[df_shops['shop_name'] == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
df_shops['city'] = df_shops['shop_name'].str.split(' ').map(lambda x: x[0])
df_shops.loc[df_shops.city == '!Якутск', 'city'] = 'Якутск'

# add distance to Moscow?

df_shops['city_code'] = LabelEncoder().fit_transform(df_shops['city'])
df_shops = df_shops[['shop_id','city_code']]

In [9]:
# Processing text features

df_items_txt = df_items[['item_id', 'item_name']]
df_items_txt['item_name'] = df_items_txt['item_name'].replace('[0-9!"\?\.)(,\+\*\[\]/:\-\'&]', ' ', regex=True)

features = 10
tfidf = TfidfVectorizer(max_features=features)
df_items_txt['item_name_len'] = df_items_txt['item_name'].map(len)  
df_items_txt['item_name_wc'] = df_items_txt['item_name'].map(lambda x: len(str(x).split(' '))) 
txtFeatures = pd.DataFrame(tfidf.fit_transform(df_items_txt['item_name']).toarray())
cols = txtFeatures.columns

for i in range(features):
    df_items_txt['item_name_tfidf_' + str(i)] = txtFeatures[cols[i]]

df_items_txt.drop(columns=['item_name'], inplace=True)


C:\Users\aimar\anaconda3\envs\kaggle\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


#### Prepare mean encodings

In [10]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

def get_all_data(df):    
    
    sales = df.copy()
    sales = pd.merge(sales, df_items[['item_id','item_category_id']], how='left', on='item_id')
    sales = pd.merge(sales, df_item_cats, how='left', on='item_category_id')
    # Create "grid" with columns
    
    # For every month we create a grid from all shops/items combinations from that month
    grid = [] 
    for block_num in sales['date_block_num'].unique():
        cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
        cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
        grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])), dtype='int32'))
    
    # Turn the grid into a dataframe
    grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
    
    # Groupby data to get shop-item-month aggregates
    gb = sales.groupby(index_cols, as_index=False).agg({'item_cnt_day':'sum'})
    gb['item_cnt_day'] = gb['item_cnt_day'].clip(0, 20)
    gb.rename(columns={'item_cnt_day': 'target'}, inplace=True)
    
    # Fix column names
    # Join it to the grid
    all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)#0.334?
    
    # Same as above but with shop-month aggregates
    gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':'sum'})
    gb['item_cnt_day'] = gb['item_cnt_day'].clip(0, 20)
    gb.rename(columns={'item_cnt_day': 'target_shop'}, inplace=True)
    all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)
    
    # Same as above but with item-month aggregates
    gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':'sum'})
    gb['item_cnt_day'] = gb['item_cnt_day'].clip(0, 20)
    gb.rename(columns={'item_cnt_day': 'target_item'}, inplace=True)
    all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)    
    
    ### Advanced features statistics for price
    # mean price
    if ENABLE_PRICE_STATS:
        gb = sales.groupby(['item_id', 'shop_id', 'date_block_num'],as_index=False).agg({'item_price':'mean'})
        gb.columns = ['item_id', 'shop_id', 'date_block_num', 'item_price_mean']
        all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'shop_id', 'date_block_num']).fillna(0)
        
        all_data['sale_total'] = all_data['target'] * all_data['item_price_mean']
        
    if enable_subtype_code_mean:
        # min/max price by 'subtype_code'
        all_data = pd.merge(all_data, sales[['item_id','item_category_id']].drop_duplicates(), how='left', on='item_id')
        all_data = pd.merge(all_data,  df_item_cats[['item_category_id', 'subtype_code']], how='left', on='item_category_id')
        gb = sales.groupby(['subtype_code', 'shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':'sum'})
        gb['item_cnt_day'] = gb['item_cnt_day'].clip(0, 20)
        gb.columns = ['subtype_code', 'shop_id', 'date_block_num', 'target_subtype_code']
        all_data = pd.merge(all_data, gb, how='left', on=['subtype_code', 'shop_id', 'date_block_num']).fillna(0)
        
        all_data.drop(columns=['item_category_id', 'subtype_code'], inplace=True)
    
    df_test_concat = df_test.drop(columns=['ID'])
    df_test_concat['date_block_num'] = 34
    all_data, TRAIN_SIZE = concat_df(all_data, df_test_concat)
    
    all_data = downcast_dtypes(all_data)
#     del grid, gb, df_test_concat, sales, df_test
#     gc.collect();
    return all_data, TRAIN_SIZE


#### Prepare historical lags item

In [11]:
shift_range = [1, 2, 3, 4, 5, 12]

def get_lags(df):
    
    # List of columns that we will use to create lags
    cols_to_rename = list(df.columns.difference(index_cols)) 
    
    df = df.copy()
    for month_shift in tqdm.tqdm(shift_range):
        train_shift = df[index_cols + cols_to_rename].copy()
        
        train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
        
        foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
        train_shift = train_shift.rename(columns=foo)
    
        df = pd.merge(df, train_shift, on=index_cols, how='left').fillna(0)
    
    del train_shift
    
    # try to delete medians
    df.drop(columns=['target_shop', 'target_item'], inplace=True)
    if ENABLE_PRICE_STATS:
        df.drop(columns=['item_price_mean', 'sale_total'], inplace=True)
        
    if enable_subtype_code_mean:    
        df.drop(columns=['target_subtype_code'], inplace=True)
    
    # List of all lagged features
    fit_cols = [col for col in df.columns if col[-1] in [str(item) for item in shift_range]] 
    # We will drop these at fitting stage
    to_drop_cols = list(set(list(df.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 
    
    # Category for each item
    item_category_mapping = df_items[['item_id','item_category_id']].drop_duplicates()
    
    df = pd.merge(df, item_category_mapping, how='left', on='item_id')
    df = pd.merge(df, df_item_cats, how='left', on='item_category_id')
    df = pd.merge(df, df_shops, how='left', on='shop_id')
    df = pd.merge(df, df_items_txt, how='left', on='item_id')
    
#     months = df_train.groupby(['date_block_num'], as_index=False)['month'].mean()
    
#     df = pd.merge(df, months, how='left', on='date_block_num')
    
#     df['month'] = ((df['date_block_num'] % 12)+1)
    
    
    
    # generating categorical interaction feature for experiment/education
    # all_data['item_category_id_city_code'] = all_data['item_category_id'].astype(str) + '_' + all_data['city_code'].astype(str)
    # all_data['item_category_id_city_code'] = LabelEncoder().fit_transform(all_data['item_category_id_city_code'])
    return downcast_dtypes(df)
    # del df_items_txt
    # gc.collect();


## Prepare train/validation validation sets

In [12]:
from sklearn.preprocessing import OneHotEncoder

def trim_12(df):
    return df[df['date_block_num'] >= 12]

# TODO should we have subtype_code as it encodes item_category_id
cat_columns = ['item_id', 
               'shop_id', 
               'item_category_id', 'type_code', 'subtype_code','city_code']#, 
               #'item_category_id_city_code]


def split_train_and_postprocess_for_linear(X_train, Y_train, X_test):
    Y_train = Y_train.reset_index(drop=True)
    
    df_all_data, train_size = concat_df(X_train, X_test)
    
    for cat_column in cat_columns:
        df_all_data[cat_column] = df_all_data[cat_column].astype('category')
             
    df_cat = df_all_data[cat_columns + ['date_block_num']]
    df_scale = df_all_data[list(set(df_all_data.columns) - set(cat_columns))]
      
    scaler = StandardScaler()
    scaler.fit(df_scale)
    
    encoder = OneHotEncoder(drop='first')
    encoder.fit(df_cat[cat_columns])
    
    train_cat, X_test_cat = divide_df(df_cat, train_size)
    X_test_cat = X_test_cat.drop(columns=['date_block_num'])
    
    max_date = train_cat['date_block_num'].max()
               
    cv_X_test_cat = train_cat[train_cat['date_block_num'] == max_date].drop(columns=['date_block_num'])
    cv_X_train_cat = train_cat[train_cat['date_block_num'] < max_date].drop(columns=['date_block_num'])
    
    train_scale, X_test_scale = divide_df(df_scale, train_size)
    cv_X_test_scale = train_scale[train_scale['date_block_num'] == max_date]
    cv_Y_test = Y_train[train_scale['date_block_num'] == max_date]
    
    cv_Y_train = Y_train[train_scale['date_block_num'] < max_date]
    cv_X_train_scale = train_scale[train_scale['date_block_num'] < max_date]
               
    # scaling
    X_test_scale = scaler.transform(X_test_scale)
    cv_X_test_scale = scaler.transform(cv_X_test_scale)
    cv_X_train_scale = scaler.transform(cv_X_train_scale)
    
    X_test_cat = encoder.transform(X_test_cat)
    X_test_scale = csr_matrix(X_test_scale)
    X_test = hstack((X_test_cat, X_test_scale))
               
    cv_X_test_cat = encoder.transform(cv_X_test_cat)
    cv_X_test_scale = csr_matrix(cv_X_test_scale)
    cv_X_test = hstack((cv_X_test_cat, cv_X_test_scale))
    
    cv_X_train_cat = encoder.transform(cv_X_train_cat)    
    cv_X_train_scale = csr_matrix(cv_X_train_scale)
    cv_X_train = hstack((cv_X_train_cat, cv_X_train_scale))
               
    return cv_X_train, cv_Y_train, cv_X_test, cv_Y_test,X_test

def preprocess_for_linear(X_train, X_test):
    df_all_data, train_size = concat_df(X_train, X_test)
    for cat_column in cat_columns:
        df_all_data[cat_column] = df_all_data[cat_column].astype('category')
            
    df_cat = df_all_data[cat_columns]
    df_scale = df_all_data[list(set(df_all_data.columns) - set(cat_columns))]
      
    scaler = StandardScaler()
    scaler.fit(df_scale)
    
    encoder = OneHotEncoder(drop='first')
    encoder.fit(df_cat)
    
    train_cat, X_test_cat = divide_df(df_cat, train_size)
    train_scale, X_test_scale = divide_df(df_scale, train_size)
               
    # scaling
    X_test_scale = scaler.transform(X_test_scale)
    X_test_cat = encoder.transform(X_test_cat)
    X_test = hstack((X_test_cat, X_test_scale))
    
    train_cat = encoder.transform(train_cat)
    train_scale = scaler.transform(train_scale)
    X_train = hstack((train_cat, train_scale))
               
    return X_train, X_test
               
def train_and_predict_lr(X, Y, X_test, model_name):
    pkl_filename = f'{model_name}.pkl'
    if pathlib.Path(pkl_filename).exists():
        print(f'Reading model {pkl_filename} from file')
        with open(pkl_filename, 'rb') as file:
            lr = pickle.load(file)
    else:
        lr = SGDRegressor()
        lr.fit(X, Y)
        print(f'Storing model {pkl_filename} in file')
        #with open(pkl_filename, 'wb') as file:
            #pickle.dump(lr, file)
    pred_lr = lr.predict(X_test)
    return pred_lr

def tune_lr_hp(X_tr, Y_tr, X_ts, Y_ts, model='SGD'):
    
    pkl_filename = f'models/{model}.pkl'
    if pathlib.Path(pkl_filename).exists():
        print(f'Reading model from file {pkl_filename}')
        with open(pkl_filename, 'rb') as file:
            return pickle.load(file)
    
    X = vstack((X_tr, X_ts))
    Y = pd.concat([Y_tr, Y_ts])    
    print(f'X: {X.shape}, Y: {Y.shape}')
    
    def cv_generator():        
        train_ind = list(range(0, X_tr.shape[0]))
        test_ind = list(range(X_tr.shape[0], X_tr.shape[0] + X_ts.shape[0]))
        
        print(f'X_tr shape: {X_tr.shape}, X_ts shape: {X_ts.shape}')
        print(f'train_ind=[{train_ind[0]}, {train_ind[-1]}], test_ind=[{test_ind[0]}, {test_ind[-1]}]')
        yield train_ind, test_ind
    
    n_HP_points_to_test = 10
    
    params = {'penalty': ['l2', 'l1', 'elasticnet'],
              'l1_ratio': sp_uniform(loc=0.05, scale=0.9),
              'alpha' : 10.0**-np.arange(1,7),
              'learning_rate' : ['constant', 'optimal', 'invscaling', 'adaptive']}
    fit_params={}
    
    gs = RandomizedSearchCV(
            estimator=SGDRegressor(), param_distributions=params, 
            n_iter=n_HP_points_to_test,
            scoring='neg_root_mean_squared_error',
            cv=cv_generator(),
            refit=True,
            random_state=314,
            verbose=False, n_jobs=2)
    gs.fit(X, Y, **fit_params)
    pd.DataFrame.from_dict(gs.cv_results_).to_csv('lr_cv.csv')
    with open(pkl_filename, 'wb') as file:
        pickle.dump(gs.best_estimator_, file)
    return gs.best_estimator_
    

#### Train lightgbm model with HP optimization and prediction
to speedup review we are storing optimized hyperparams, model training should not take much time

In [13]:
def regularize(all_data):
    X = X.copy()
    X['target'] = Y
    skf = StratifiedKFold(n_splits=5)
    print('X.index')
    print(X.index)
    for train_index, val_index in skf.split(X, X[regularize_column]):
        df_train, df_val = X.iloc[train_index], X.iloc[val_index]
        group = df_train.groupby(regularize_column).target.mean()
        X.loc[val_index, target_enc] = X.loc[val_index, regularize_column].map(group)
    X[target_enc].fillna(0.3343, inplace=True)
    return X[target_enc]

def get_cv_time_series_split(all_data, n_splits=4):
    min_date = all_data['date_block_num'].min()
    max_date = all_data['date_block_num'].max()
    print(f'min_date: {min_date}, max_date: {max_date}')
    months = random.sample(range(min_date+9, max_date), n_splits)
    for month in months:        
        train_index = all_data[(all_data['date_block_num'] < month)].index
        test_index = all_data[all_data['date_block_num'] == month].index
        print(f'time series cv split month: {month}, train size: {len(train_index)}, test size: {len(test_index)}')
        yield train_index.values, test_index.values

def optimize_lgb_parameters(X, Y, X_val, Y_val, month=''):
    params_file = f'models/lgb_parameters_{month}.json'
    if pathlib.Path(params_file).exists():
        with open(params_file) as f:
            params = json.load(f)
            print('Lgbm parameters read from file.')
            print(params)
        return params
    else:
        print('Lgbm HP optimization...')
#         lgb_params ={'num_leaves': sp_randint(6, 50), 
#              'min_child_samples': sp_randint(100, 500), 
#              'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
#              'subsample': sp_uniform(loc=0.2, scale=0.8), 
#              'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
#              'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
#              'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
#              'bagging_fraction': sp_uniform(loc=0.5, scale=0.45),
#              'feature_fraction': sp_uniform(loc=0.5, scale=0.45),
#              'min_data_in_leaf': sp_randint(0, 300),
# #                 'learning_rate': [0.03]
#                     }
        lgb_params ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}
        fit_params={"early_stopping_rounds":30, 
                    "eval_metric" : 'rmse', 
                    "eval_set" : [(X_val, Y_val)],
                    'eval_names': ['valid'],
                    'verbose': 100,
#                     'categorical_feature': 'auto'
                   }
        
        n_HP_points_to_test = 50
        
        clf = lgb.LGBMRegressor(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=-1, 
                                n_estimators=50)
        X = X.reset_index(drop=True)
        Y = Y.reset_index(drop=True)
        cv_generator = get_cv_time_series_split(X)
        gs = RandomizedSearchCV(
            estimator=clf, param_distributions=lgb_params, 
            n_iter=n_HP_points_to_test,
            scoring='neg_root_mean_squared_error',
            cv=cv_generator,
            refit=True,
            random_state=314,
            verbose=False)
        gs.fit(X, Y, **fit_params)
        print(f'Best score reached: {gs.best_score_} with params: {gs.best_params_}')
        print(f'Feature importance HP optimization: {gs.best_estimator_.feature_importances_}')
#         print(f'Model: {gs.best_estimator_.model_to_string()}')
        params = gs.best_params_
        with open(params_file, 'w') as f:
            json.dump(params, f)
        pd.DataFrame.from_dict(gs.cv_results_).to_csv(f'results/lgb_csv_{month}.csv')
        return params

def train_and_predict_lgbm(X, Y, X_test, params, month=''):
    model_file = f'models/lgb_{month}.txt'
    if pathlib.Path(model_file).exists():
        print(f'reading lgb model from {model_file}')
        model = lgb.Booster(model_file=model_file)
    else:
        print(f'training for {model_file} by parameters')
        model = lgb.train(params, lgb.Dataset(X, label=Y), 100)
        model.save_model(model_file)
        
    pred_lgb = model.predict(X_test)
    return pred_lgb 

In [14]:
all_data, TRAIN_SIZE = get_all_data(df_train)
all_data = get_lags(all_data)

# ???
# for cat_column in cat_columns:
#     all_data[cat_column] = all_data[cat_column].astype('category')

train, X_test = divide_df(all_data, TRAIN_SIZE)

# Don't use old data from year 2013
train = trim_12(train) 

item_ids_subset = get_items_subset(train)
train = train[item_ids_subset]

X_train = train.drop(columns=['target'])
Y_train = train['target']

max_date = X_train['date_block_num'].max()

# IMPORTANT this is holdout validation test set
cv_X_test = X_train[X_train['date_block_num'] == max_date]
cv_Y_test = Y_train[X_train['date_block_num'] == max_date]

cv_Y_train = Y_train[X_train['date_block_num'] < max_date]
cv_X_train = X_train[X_train['date_block_num'] < max_date]

del df_train, all_data
gc.collect();

## Ensembling

In [15]:
meta_result = {}
def get_meta_featues_train(X_tr, Y_tr, X_ts):

    dates_train = X_tr['date_block_num']
    
    meta_dates = [27, 28, 29, 30, 31, 32, 33]

#     meta_dates = [27]
    
    dates_train_level2 = dates_train[dates_train.isin(meta_dates)]
    
    # That is how we get target for the 2nd level dataset
    y_train_level2 = Y_tr[dates_train.isin(meta_dates)]
    
    # And here we create 2nd level feeature matrix, init it with zeros first
    X_train_level2 = np.zeros([y_train_level2.shape[0], 2])
    
    # Fit N diverse models on those M chunks and predict for the chunk M+1. 
    # Then fit those models on first M+1 chunks and predict for chunk M+2 and so on, until you hit the end.
    for cur_block_num in meta_dates:
        print(f'Ensembling month {cur_block_num}')
        
        '''
            1. Split `X_train` into parts
               Remember, that corresponding dates are stored in `dates_train` 
            2. Fit linear regression 
            3. Fit LightGBM and put predictions          
            4. Store predictions from 2. and 3. in the right place of `X_train_level2`. 
               You can use `dates_train_level2` for it
               Make sure the order of the meta-features is the same as in `X_test_level2`
        '''      
        
        train_x = X_tr.loc[dates_train < cur_block_num]
        train_y = Y_tr[dates_train < cur_block_num]
        
        test_x = X_tr.loc[dates_train == cur_block_num]
        test_y = Y_tr[dates_train == cur_block_num]
        
        cv_X_train_lr, cv_Y_train_lr, cv_X_test_lr, cv_Y_test_lr, X_test_lr \
            = split_train_and_postprocess_for_linear(train_x, train_y, test_x)
        
        be = tune_lr_hp(cv_X_train_lr, cv_Y_train_lr, cv_X_test_lr, cv_Y_test_lr, model=f'lr_meta_{cur_block_num}')
        pred_lr = be.predict(X_test_lr)
        lr_rmse = mean_squared_error(test_y, pred_lr, squared=False)
        meta_result[f'lr_rmse_{cur_block_num}'] = [lr_rmse]
        
        cv_max_date = train_x['date_block_num'].max()

        # IMPORTANT this is holdout validation test set
        cv_X_ts = train_x[train_x['date_block_num'] == cv_max_date]
        cv_Y_ts = train_y[train_x['date_block_num'] == cv_max_date]
        
        cv_Y_tr = train_y[train_x['date_block_num'] < cv_max_date]
        cv_X_tr = train_x[train_x['date_block_num'] < cv_max_date]        
        
        lgb_parameters = optimize_lgb_parameters(cv_X_tr, cv_Y_tr, cv_X_ts, cv_Y_ts, month=cur_block_num)
        pred_lgb = train_and_predict_lgbm(train_x, train_y, test_x, lgb_parameters, month=cur_block_num)
        lgb_rmse = mean_squared_error(test_y, pred_lgb, squared=False)
        meta_result[f'lgb_rmse_{cur_block_num}'] = [lgb_rmse]
                
        X_train_level2[dates_train_level2==cur_block_num, 0] = pred_lr
        X_train_level2[dates_train_level2==cur_block_num, 1] = pred_lgb
        
    
    best_alpha = get_best_alpha(X_train_level2, y_train_level2)
    meta_result['best_alpha'] = [best_alpha]
    test_preds = get_mix(best_alpha, np.c_[pred_lr, pred_lgb])
    rmse_mix = mean_squared_error(test_y, test_preds)
    meta_result['rmse_mix'] = [rmse_mix]
    
    # Use all train data to fit models and get predictions for test.
    be.fit(X_tr, Y_tr)
    pred_lr = be.predict(X_ts)
    pred_lgb = train_and_predict_lgbm(X_tr, Y_tr, X_ts, lgb_parameters, 33)
    X_test_level2 = np.c_[pred_lr, pred_lgb] 
    
    return X_train_level2, y_train_level2, X_test_level2

In [ ]:
X_train_level2, y_train_level2, X_test_level2 = get_meta_featues_train(X_train, Y_train, X_test)
del train
lr = LinearRegression()
lr.fit(X_train_level2, y_train_level2)
train_preds = lr.predict(X_train_level2)
rmse_train_stacking = mean_squared_error(y_train_level2, train_preds)
meta_result['rmse_train_stacking'] = [rmse_train_stacking]
meta_result['cv_fraction'] = [cv_fraction]

test_preds = lr.predict(X_test_level2)
get_result_df(test_preds).to_csv('result_ens.csv', index=False)

if pathlib.Path('results/meta_features_results.csv').exists():
    metrics = pd.read_csv('results/meta_features_results.csv')
else:
    metrics = pd.DataFrame()
metrics = pd.concat([metrics, pd.DataFrame(data={k: [round(v[0], 5)] for k,v in meta_result.items()})], ignore_index=True)
metrics.to_csv('results/meta_features_results.csv', index=False)


Ensembling month 27
Reading model from file models/lr_meta_27.pkl
Lgbm parameters read from file.
{'colsample_bytree': 0.46716991724263107, 'min_child_samples': 439, 'min_child_weight': 1, 'num_leaves': 46, 'reg_alpha': 1, 'reg_lambda': 50, 'subsample': 0.20651472131008397}
reading lgb model from models/lgb_27.txt
Ensembling month 28
X: (5068102, 15736), Y: (5068102,)
X_tr shape: (4810730, 15736), X_ts shape: (257372, 15736)
train_ind=[0, 4810729], test_ind=[4810730, 5068101]
Lgbm HP optimization...
min_date: 12, max_date: 26
time series cv split month: 24, train size: 3939517, test size: 306950
time series cv split month: 25, train size: 4246467, test size: 284491
time series cv split month: 21, train size: 2963799, test size: 329368
time series cv split month: 22, train size: 3293167, test size: 316100
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[11]	valid's rmse: 0.927842
Training until validation scores don't improve for 30 rounds

Early stopping, best iteration is:
[15]	valid's rmse: 0.925273
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[19]	valid's rmse: 0.920322
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[28]	valid's rmse: 0.920211
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[47]	valid's rmse: 0.916139
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[14]	valid's rmse: 0.917125
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[25]	valid's rmse: 0.901392
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[20]	valid's rmse: 0.902033
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[25]	valid's rmse: 0.898408
Training until va

Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[25]	valid's rmse: 0.902167
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[14]	valid's rmse: 0.926324
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[21]	valid's rmse: 0.910694
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[21]	valid's rmse: 0.910659
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[24]	valid's rmse: 0.902511
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[11]	valid's rmse: 0.924551
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[19]	valid's rmse: 0.907569
Training until validation scores don't improve for 30 rounds
Did not meet early 

Early stopping, best iteration is:
[19]	valid's rmse: 0.910307
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[30]	valid's rmse: 0.910025
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[30]	valid's rmse: 0.902681
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[14]	valid's rmse: 0.924263
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[26]	valid's rmse: 0.914703
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[30]	valid's rmse: 0.909115
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[50]	valid's rmse: 0.904955
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[14]	valid's rmse: 0.925252
Trai

## Final train/validation/prediction setup

# ROUND I 
# FINDING OPTIMAL PARAMETERS AND PARAMETER ALPHA FOR MIXING UP RESULTS OF LBG AND LINEAR MODELS

lgb_parameters = optimize_lgb_parameters(cv_X_train, cv_Y_train, cv_X_test, cv_Y_test)

# Use holdout validation test set to find optimal alpha

pred_tree = train_and_predict_lgbm(cv_X_train, cv_Y_train, cv_X_test, lgb_parameters, 'lgb_model_validation')
lgb_r2 = r2_score(cv_Y_test, pred_tree)
lgb_rmse = mean_squared_error(cv_Y_test, pred_tree, squared=False)

cv_X_train_lr, cv_Y_train_lr, cv_X_test_lr, cv_Y_test_lr, X_test_lr \
    = split_train_and_postprocess_for_linear(X_train, Y_train, X_test)

# pred_lr = train_and_predict_lr(cv_X_train_lr, cv_Y_train_lr, cv_X_test_lr, 'lr_model_full')
be = tune_lr_hp(cv_X_train_lr, cv_Y_train_lr, cv_X_test_lr, cv_Y_test_lr)
pred_lr = be.predict(cv_X_test_lr)
lr_r2 = r2_score(cv_Y_test_lr, pred_lr)
lr_rmse = mean_squared_error(cv_Y_test_lr, pred_lr, squared=False)

X_train_level2 = np.c_[pred_tree, pred_lr]

best_alpha = get_best_alpha(X_train_level2, cv_Y_test)
mix = get_mix(best_alpha, X_train_level2)
mix_r2 = r2_score(cv_Y_test, mix)
mix_rmse = mean_squared_error(cv_Y_test, mix, squared=False)

results = {'cv_fraction':cv_fraction, 
           'lr_rmse': lr_rmse, 'lr_r2': lr_r2, 
           'lgb_rmse': lgb_rmse, 'lgb_r2': lgb_r2, 
           'mix_rmse': mix_rmse, 'mix_r2': mix_r2, 
           'alfa': best_alpha}
print(results)
del cv_X_train, cv_Y_train, cv_X_test, cv_Y_test, cv_X_train_lr, cv_Y_train_lr, cv_X_test_lr, cv_Y_test_lr,\
    pred_lr, pred_tree
gc.collect();

# Store and display validation metruics results
results_metrics = 'results_metrics.csv'
columns=['lr_rmse', 'lr_r2', 'lgb_rmse', 'lgb_r2', 'mix_rmse', 'mix_r2', 'alfa']

if pathlib.Path(results_metrics).exists():
    metrics = pd.read_csv(results_metrics)
else:
    metrics = pd.DataFrame(columns=['cv_fraction'] + columns)

metrics = pd.concat([metrics, pd.DataFrame(results, index=[len(metrics)+1])])   
metrics.to_csv(results_metrics, index=False)


for cv_fraction in metrics['cv_fraction'].unique():
    ax =plt.axes([0, 0, 2, 2])
    for col in columns:
        sub_metrix = metrics[metrics['cv_fraction'] == cv_fraction]
        ax.plot(sub_metrix.index, sub_metrix[col], marker='.', linestyle='-', ms=12, label=col)
        for i,j in zip(sub_metrix.index,metrics[col]):
            ax.annotate(str(round(j,4)),xy=(i,j))
        plt.xlabel(f'cv_fraction: {cv_fraction}')

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.grid(True)
    plt.show()


# ROUND II
# TRAIN MODELS ON COMPLETE DATA SET AND GETTING PREDICTIONS FOR SUBMITING
X_train_lr, X_test_lr = preprocess_for_linear(X_train, X_test)
# pred_lr = train_and_predict_lr(X_train_lr, Y_train, X_test_lr, 'lr_model_full')
be.fit(X_train_lr, Y_train)
pred_lr = be.predict(X_test_lr)

del X_train_lr, X_test_lr
gc.collect();

pred_tree = train_and_predict_lgbm(X_train, Y_train, X_test, lgb_parameters, 'lgb_model_train')

X_train_level2 = np.c_[pred_tree, pred_lr]

result = get_mix(best_alpha, X_train_level2)

get_result_df(result).to_csv('result_lgb+lr.csv', index=False)
# Your public and private LB scores are: 0.962927 and 0.963253.
get_result_df(pred_tree).to_csv('result_lgb.csv', index=False)
print("DONE!")


In [ ]:
round(time.time() - start)/60